In [1]:
import os

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx',
 'crawler_2.xlsx',
 'crawler_3.xlsx',
 'crawler_4.xlsx',
 'crawler_5.xlsx',
 'crawler_6.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_ = input_.drop_duplicates(['JOIN_MPNTCPC'],
                                keep='first',
                                ignore_index=True)

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['JOIN_MPNTCPC'].tolist()

总数量：2357


['MEVOTECH;18750198;7580;10362404',
 'MEVOTECH;GK3093;7580;5559111',
 'MEVOTECH;GK3124;7580;5559114',
 'MEVOTECH;GK3150;7580;5559135',
 'MEVOTECH;GK3173;7580;5559168',
 'MEVOTECH;GK3174;7580;5559171',
 'MEVOTECH;GK3196;7580;5559192',
 'MEVOTECH;GK3197;7580;5559195',
 'MEVOTECH;GK3200;7580;5559204',
 'MEVOTECH;GK3201;7580;5559207',
 'MEVOTECH;GK3202;7580;5559210',
 'MEVOTECH;GK446;7580;5559216',
 'MEVOTECH;GK447;7580;5559219',
 'MEVOTECH;GK5252;7580;5559261',
 'MEVOTECH;GK5254;7580;5559267',
 'MEVOTECH;GK5255;7580;5559270',
 'MEVOTECH;GK5315;7580;5559333',
 'MEVOTECH;GK5319;7580;5559336',
 'MEVOTECH;GK5334;7580;5559363',
 'MEVOTECH;GK5342;7580;5559369',
 'MEVOTECH;GK5345;7580;5559372',
 'MEVOTECH;GK6217;7580;5559525',
 'MEVOTECH;GK6262;7580;5559573',
 'MEVOTECH;GK6428;7580;5559777',
 'MEVOTECH;GK6599;7580;5559912',
 'MEVOTECH;GK6600;7580;5559915',
 'MEVOTECH;GK6602;7580;5559918',
 'MEVOTECH;GK6628;7580;5559924',
 'MEVOTECH;GK6629;7580;5559927',
 'MEVOTECH;GK6630;7580;5559930',
 'MEVOTEC

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

payload = {'partData': {'listing_data_essential': {'parttype': input_.loc[a, 'Type Code'],
                                                   'partkey': input_.loc[a, 'Part Code']},
                        'listing_data_supplemental': {'partnumber': input_.loc[a, 'Part Number'],
                                                      'catalogname': input_.loc[a, 'Manufacturer']}}}

data = {'func': 'getbuyersguide',
        'payload': json.dumps(payload),
        'api_json_request': '1'}

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()
    
            if 'redirect_to_url' not in json_ and json_['buyersguidepieces']['title'].startswith('Buyer\'s Guide :') and 'collected_javascript' in json_:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['buyersguidepieces']['body'], 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="buyersguide-nested">
   <div style="padding: .5em;">
    <table class="nobmp" style="width: 100%;">
     <tr>
      <td class="altrow-a-0">
       MITSUBISHI
      </td>
      <td class="altrow-a-0">
       LANCER
      </td>
      <td class="altrow-a-0">
       2003-2006
      </td>
     </tr>
    </table>
    <div>
     Please refer to catalog for application details.
    </div>
   </div>
  </div>
 </body>
</html>



In [9]:
list_vehicle = []
len(html.xpath('//p/text()')) != 0 and html.xpath('//p/text()')[0] == 'No applications found.'

False

In [10]:
list_tr = html.xpath('//tr')

list_tr

[<Element tr at 0x12b2a1480>]

In [11]:
len(list_tr) == 0

False

In [12]:
for i, tr in zip(range(len(list_tr)), list_tr):
    vehicle = [text.strip() for text in tr.xpath('./td/text()')]
    list_vehicle.append(' '.join(vehicle))

list_vehicle

['MITSUBISHI LANCER 2003-2006']

In [13]:
df_temp = pd.DataFrame([{'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC'],
                         'Vehicle': '\n'.join(sorted(list_vehicle))}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,JOIN_MPNTCPC,Vehicle
0,MEVOTECH;18750198;7580;10362404,MITSUBISHI LANCER 2003-2006


In [14]:
crawler_status = 'ok'

crawler_status

'ok'

In [15]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [16]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['JOIN_MPNTCPC'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,JOIN_MPNTCPC,Vehicle
0,MEVOTECH;18750198;7580;10362404,MITSUBISHI LANCER 2003-2006


In [17]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
